In [1]:
import numpy as np     
import pandas as pd     
from scipy import stats
import seaborn as sns
import math

%matplotlib inline

In [2]:
x1_size = 100
x1 = stats.chi2(2).rvs(x1_size)

x1_mean = np.mean(x1)
x1_var = np.var(x1)
x1_std = math.sqrt(x1_var) 

print(f"X1: mean = {x1_mean:.3f}; var = {x1_var:.3f}; std = {x1_std:.3f}; size = {x1_size:.3f}")

X1: mean = 2.154; var = 6.132; std = 2.476; size = 100.000


In [3]:
x2_size = 100
x2 = stats.chi2(2).rvs(x2_size)

x2_mean = np.mean(x2)
x2_var = np.var(x2)
x2_std = math.sqrt(x2_var) 

print(f"X2: mean = {x2_mean:.3f}; var = {x2_var:.3f}; std = {x2_std:.3f}; size = {x2_size:.3f}")

X2: mean = 2.222; var = 4.368; std = 2.090; size = 100.000
